In [1]:
import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d, griddata
from itertools import product
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
import glob
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu_r')


pd.set_option('display.max_rows', 200)

### Load Recast Data

In [2]:
recastData = pd.read_pickle('../Recast/pp2ChiChi_monojet.pcl')
mstop = '$m_{\tilde t}$'
mchi = '$m_{\tilde \chi_1^0}$'

### Get all model points

In [3]:
models = []
mCols = ['Coupling','Mode',mstop,mchi]
for row in recastData[mCols].values:
    m = dict(zip(mCols,row.tolist()))
    if m not in models:
        models.append(m)
print('%i models loaded' %len(models))

36 models loaded


### Get model parameters

In [4]:
parameters = {label : list(set([m[label] for m in models])) for label in mCols}
for k,v in parameters.items():
    if len(v) == 1:
        parameters[k] = v[0]
print(parameters)

textstr = '\n'.join([r'%s = %s' %(k,v) for k,v in parameters.items() if k not in [mstop,mchi]])
print(textstr)

{'Coupling': 'Stop', 'Mode': 'DM+QCDjets', '$m_{\tilde t}$': [800.0, 900.0, 1000.0, 500.0, 600.0, 700.0], '$m_{\tilde \\chi_1^0}$': [100.0, 200.0, 300.0, 400.0, 50.0, 500.0]}
Coupling = Stop
Mode = DM+QCDjets


In [5]:
# Remove duplicated data from other datasets
recastData = recastData[recastData['Data-takingperiod'] == 2017]

In [6]:
kfactor = 1.0
lmuData = np.log10(recastData['$\mu^{UL}_{obs}$']/kfactor)
rValues = 1/((recastData['$\mu^{UL}_{obs}$']/kfactor))

KeyError: '$\\mu^{UL}_{obs}$'

In [ ]:
fig = plt.figure(figsize=(7,5))
y = recastData[mstop]
x = recastData[mchi]
plt.scatter(x=x,y=y,vmin=0.0,vmax=2.0,
            c=rValues,cmap=cm,s=60)
cbar = plt.colorbar(label=r'$r$-value')
plt.xlabel(r'$m_{\tilde \chi}$ (GeV)')
plt.ylabel(r'$m_{\tilde t}$ GeV)')
# plt.yscale('log')
# plt.ylim(-10,1000)
# plt.xlim(0,3000)
plt.title('kfactor = %1.2f' %kfactor)
plt.show()

### Interpolate for a denser grid

In [ ]:
pts = np.array([x for x in product(np.arange(10.,500.,10.),np.arange(500.,1100.,25.))])
rValues.fillna(0.,inplace=True)
rPts = griddata(list(zip(recastData[mchi],recastData[mstop])),
                  rValues,pts)
pts = pts[~np.isnan(rPts)]
rPts = rPts[~np.isnan(rPts)]

In [ ]:
fig = plt.figure(figsize=(7,5))
y = recastData[mstop]
x = recastData[mchi]
plt.scatter(x=x,y=y,vmin=0.0,vmax=2.0,
            c=rValues,cmap=cm,s=60)
cbar = plt.colorbar(label=r'$r$-value')
cs = plt.tricontour(pts[:,0],pts[:,1],rPts,levels=[1.0],
               linewidths=2,linestyles='solid',colors='black')
plt.xlabel(r'$m_{\tilde \chi}$ (GeV)')
plt.ylabel(r'$m_{\tilde t}$ GeV)')
# plt.yscale('log')
plt.ylim(500,1200)
# plt.xlim(150,550)
plt.title('kfactor = %1.2f' %kfactor)

plt.show()

In [ ]:
np.savetxt('monojet_curve_dmeft.csv',(cs.allsegs[0][0][:,1],cs.allsegs[0][0][:,0]),fmt='%1.4e',
           header='# mstop[GeV],mchi[GeV]',delimiter=',')

In [ ]:
fig = plt.figure(figsize=(7,5))
y = recastData[mstop]
x = recastData[mchi]
plt.scatter(x=y,y=y-x,vmin=0.0,vmax=2.0,
            c=rValues,cmap=cm,s=60)
cbar = plt.colorbar(label=r'$r$-value')
cs = plt.tricontour(pts[:,1],pts[:,1]-pts[:,0],rPts,levels=[1.0],
               linewidths=2,linestyles='solid',colors='black')
plt.ylabel(r'$\Delta M$ (GeV)')
plt.xlabel(r'$m_{\tilde t}$ GeV)')
plt.yscale('log')
plt.xlim(500,1200)
# plt.xlim(150,550)
plt.title('kfactor = %1.2f' %kfactor)

plt.show()